In [ ]:
##

In [1]:
import matplotlib.pyplot as plt
import matplotlib
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd
from scipy import signal as sp
import math
import cv2
import csv
from natsort import natsorted
from surgeon_recording.synchronization import Synchro
#%matplotlib widget

from plotly import subplots

import plotly.graph_objects as go
import plotly.express as px
 

In [2]:
reader = Reader()

In [3]:
#data_folder = join('..', 'data_exp')

subject_list = ['7','4','10','12']

segment_list = ['500','510','530','540','541','542','560']


blacklist = []
#blacklist = ['1/run2', '3/run3', '3/run5']
#blacklist = ['7/run2']
#blacklist = [join(data_folder, x) for x in blacklist]

#specialty_keys = ["neuro", "plastic", "general", "none", "maxilo"]
expertise_keys = ["novice", "expert"]

sensors = ['optitrack', 'emg', 'TPS_calibrated']


tools_tweezer = ['tweezer_x', 'tweezer_y', 'tweezer_z']
tools_scissors = ['scissors_x', 'scissors_y', 'scissors_z'] 
#tools_verif_holder = ['needle_holder_x', 'needle_holder_y', 'needle_holder_z'] #subject 1
tools_verif_holder = ['needle_holder2_x', 'needle_holder2_y', 'needle_holder2_z'] #subject 2 to 12

path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_exp'                 #lasa computer


In [4]:
data_segments = {}
mean_segments = {}
emg_segment_subject={}
tps_segment_subject={}
abs_diff_tps = {}
range_raw_tps_subject = {}

recording_session = {}
run_list = {}

headers = {}
file_list = {}

#    if (subject == '7') or (subject =='2'):
run_list['7'] = ['1','3','4','6','7']
run_list['2'] = ['1','3','4','6','7']
#    if (subject == '4'):
run_list['4'] = ['1','3','4','5','6']
#    if (subject == '10'):
run_list['10'] = ['1','2','4','5','6']
#    if (subject == '10'):
run_list['12'] = ['1','2','3']


recording_session['1'] = '70622'
recording_session['2'] = '70622'
recording_session['3'] = '80622'
recording_session['4'] = '80622'
recording_session['5'] = '80622'
recording_session['6'] = '90622'
recording_session['7'] = '90622'
recording_session['8'] = '90622'
recording_session['9'] = '90622'
recording_session['10']= '100622'
recording_session['11']= '100622'
recording_session['12']= '100622'



for subject in subject_list:
    data_segments[subject] = {}
    mean_segments[subject]= {}
    file_list[subject] = {}
    emg_segment_subject[subject] = {}
    tps_segment_subject[subject] = {}
    
    abs_diff_tps[subject] = {}
    range_raw_tps_subject[subject] = {}
                     
    
        
    for run in run_list[subject]:
        data_segments[subject][run] = {}
        mean_segments[subject][run]= {}
        file_list[subject][run] = {}
        abs_diff_tps[subject][run] = {}
        
        
        for sensor in sensors:
            data_segments[subject][run][sensor]= {}
            mean_segments[subject][run][sensor]= {}
            file_list[subject][run][sensor] = {}

            for segment in segment_list:
                data_segments[subject][run][sensor][segment]= {}
                file_list[subject][run][sensor][segment] = []
                mean_segments[subject][run][sensor][segment]= {}
                
                emg_segment_subject[subject][segment]={}
                tps_segment_subject[subject][segment]={}
                abs_diff_tps[subject][run][segment] = {}
                range_raw_tps_subject[subject][segment] = {}

                

for sensor in sensors:
    headers[sensor] = []
                
            

In [5]:
for subject in subject_list:   
    
    
    for run in run_list[subject]:
    
        data_folder = join('..', 'data_exp', recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION' )
        
        
        for sensor in sensors:
            
            for segment in segment_list:
                file_list[subject][run][sensor][segment] = join(data_folder, 'segment_nb_' + segment + '_' + sensor +'.csv')
                
                file = open(file_list[subject][run][sensor][segment])
                reader = csv.DictReader(file)
                headers[sensor] = reader.fieldnames
                
                
                if sensor == 'optitrack':
                    tools = headers[sensor][3:]
                if sensor == 'TPS_calibrated':
                    fingers_cal = headers[sensor][15:-1]
                if sensor == 'emg':
                    channels = headers['emg'][4:]
                    
                data = pd.read_csv(file_list[subject][run][sensor][segment])
                
                
                if sensor == 'emg':
                    for channel in channels:                    
                        data_segments[subject][run][sensor][segment][channel]= data[channel]
                        data_segments[subject][run][sensor][segment]['rel_time'] = data['relative_time']
                
                if sensor == 'TPS_calibrated':
                    for finger in fingers_cal:                    
                        data_segments[subject][run][sensor][segment][finger]= data[finger]
                        data_segments[subject][run][sensor][segment]['rel_time'] = data[' relative_time']
                    
                if sensor == 'optitrack':
                    for tool in tools:
                        data_segments[subject][run][sensor][segment][tool]= data[tool]
                        data_segments[subject][run][sensor][segment]['rel_time'] = data['relative_time']              
                    
                    
                
                
                

## EMG processing

In [6]:
fs = 1000
fny = fs/2
fco = 25
rms_window = 20

sensor = 'emg'

for subject in subject_list:   
        for segment in segment_list:
            for channel in channels:  
                emg_segment_subject[subject][segment][channel]=[]




for subject in subject_list:   
    for run in run_list[subject]:
        for segment in segment_list:
            
            for channel in channels:  
                
            #channel = 'emg0'
                data_current = np.array(data_segments[subject][run][sensor][segment][channel])
                # Remove all occurrences of elements with value above 1000 (outliers)
                data_current = data_current[data_current < 1000]
                # remove missing data points
                data_current = data_current[data_current != 0]  

                #data_analysis.processed_emg_data
                N = 4
                plt.ion
                [b, a] = sp.butter(N, 1.16*fco/fny)
                rec_signal = abs(data_current)
                butt_signal = sp.filtfilt(b, a, rec_signal)

                rms_env = np.sqrt(np.convolve(np.square(rec_signal), np.ones(rms_window)/rms_window, mode='same'))

                start_zoom = 1
                stop_zoom = 1000
                df = pd.DataFrame({

                    'rectified signal':rec_signal[start_zoom:stop_zoom],
                    'Butterworth':butt_signal[start_zoom:stop_zoom],
                    'RMS':rms_env[start_zoom:stop_zoom]
                })
                #fig = px.line([rec_signal, butt_signal])
                fig = px.line(df)

                # showing the plot
                #fig.show()

                if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION', 'emg_plots' )):
                    os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION', 'emg_plots'))
                fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION','emg_plots', 'segment' + segment + '_' + channel +'_zoom.png'))


                df = pd.DataFrame({

                    'line2':rec_signal,
                    'line1':butt_signal,
                    'line3':rms_env
                })
                #fig = px.line([rec_signal, butt_signal])
                fig = px.line(df)

                # showing the plot
                #fig.show()
                fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION','emg_plots', 'segment' + segment + '_' + channel +'_tot.png'))
                
                
                mean_segments[subject][run][sensor][segment][channel] = np.mean(butt_signal) #valeurs moyennes pour un subject sur ce run pour chaque channel
                 
                emg_segment_subject[subject][segment][channel].append(butt_signal) #valeurs emg pour 1 sujet, plusieurs runs, 1 channel

            

### plot emg means on colormap per run per subject AND per subject for all runs

In [37]:
sensor = 'emg'
for subject in subject_list:   
    for segment in segment_list:
        
            fig = go.Figure()
            x_values = []
            y_values_tot = []
            y_min = 0
            y_max = 50
            y_limit = 1.2

        
            for i, channel in enumerate(channels):
                x_values.append(i)
                
                tot_signal = np.concatenate(emg_segment_subject[subject][segment][channel])
                y_values_tot.append(np.mean(tot_signal))
            
            y_values_tot = (np.array(y_values_tot) - y_min) / (y_max - y_min)

            fig.add_trace(go.Scatter(
                x=x_values,
                y=y_values_tot,
                marker=dict(
                    size=16,
                    cmax=1,
                    cmin=0,
                    color=y_values_tot,
                    colorbar=dict(
                        title='Colorbar, 1=' + str(y_max) +' [uV]'
                    ),
                    colorscale="rainbow"
                    
                ),
                mode="markers"))
            
            fig.update_layout(title='Mean value of EMG enveloppes over runs, one subject',
                   xaxis_title='EMG channels',
                   yaxis_title='EMG data',
                   width=1025,
                   height=500,
                   yaxis_range=[0,y_limit])
            
            if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, 'PLOTS')):
                    os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, 'PLOTS'))

            fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, 'PLOTS', 'colormap_subject'+ subject + '_segment' + segment + '_' +'.png'))
            
            for run in run_list[subject]:
                fig = go.Figure()
                x_values = []
                y_values = []

                for i, channel in enumerate(channels):
                    x_values.append(i)
                    y_values.append(mean_segments[subject][run][sensor][segment][channel])

                y_values = (np.array(y_values) - y_min) / (y_max - y_min)

                fig.add_trace(go.Scatter(
                    x=x_values,
                    y=y_values,
                    marker=dict(
                        size=16,
                        cmax=1,
                        cmin=0,
                        color=y_values,
                        colorbar=dict(
                            title='Colorbar, 1=' + str(y_max) +' [uV]'
                        ),
                        colorscale="rainbow"
                    ),
                    mode="markers"))
                
                fig.update_layout(title='Mean value of EMG enveloppes, one run',
                   xaxis_title='EMG channels',
                   yaxis_title='EMG data',
                   width=1025,
                   height=500,
                   yaxis_range=[0,y_limit])

                if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, run, 'PLOTS')):
                        os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, run, 'PLOTS'))

                fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'PLOTS', 'colormap_subject'+ 'subject' + '_segment' + segment + '_run' + run +'.png'))
            

### colormap per expertise


In [38]:
sensor = 'emg'
expert_subjects = ['4','10']
novice_subjects = ['7', '12']
category = ['novice', 'expert']

y_min = 0
y_max = 50
y_limit=1.2

subject_expertise = {
    'expert': expert_subjects,
    'novice': novice_subjects
}
emg_segment_subject_expertise = {}
for cat in category: 
    emg_segment_subject_expertise[cat] = {}
    for segment in segment_list:
        emg_segment_subject_expertise[cat][segment] = {}
        for channel in channels:
            emg_segment_subject_expertise[cat][segment][channel] = []
    
    
for cat in category:               
    for subject in subject_expertise[cat]:   
        for segment in segment_list:
                for channel in channels:
                    emg_segment_subject_expertise[cat][segment][channel].append(emg_segment_subject[subject][segment][channel]) #met a la suite les valeurs de donnes pour les runs des sujets avec la meme expertise, pour 1 channel

for cat in category:               
    for subject in subject_expertise[cat]:   
        for segment in segment_list:
                fig = go.Figure(layout_yaxis_range=[0,1])
                x_values = []
                y_values_tot = []
                
                
                
                for i, channel in enumerate(channels):
                    x_values.append(i)

                    tot_signal = np.concatenate(np.concatenate(emg_segment_subject_expertise[cat][segment][channel]))
                    y_values_tot.append(np.mean(tot_signal))

                y_values_tot = (np.array(y_values_tot) - y_min) / (y_max - y_min)

                fig.add_trace(go.Scatter(
                    x=x_values,
                    y=y_values_tot,
                    marker=dict(
                        size=16,
                        cmax=1,
                        cmin=0,
                        color=y_values_tot,
                        colorbar=dict(
                            title='Colorbar, 1=' + str(y_max) +' [uV]'
                        ),
                        colorscale="rainbow",
                        
                    ),
                    mode="markers"),
                    )
                fig.update_layout(title='Mean value of EMG enveloppes for' + cat + ' subjects',
                   xaxis_title='EMG channels',
                   yaxis_title='EMG data',
                   width=1025,
                   height=500,
                   yaxis_range=[0,y_limit])


                if not os.path.exists(join(path_to_data_folder, 'plots_expertise' )):
                        os.makedirs(join(path_to_data_folder,'plots_expertise'))

                fig.write_image(join('..', 'data_exp', 'plots_expertise', 'colormap_segment' + segment + '_' + cat +'.png'))
                


## TPS processing

In [11]:
sensor = 'TPS_calibrated'

for subject in subject_list:   
        for segment in segment_list:
            for finger in fingers_cal:  
                tps_segment_subject[subject][segment][finger]=[]
                range_raw_tps_subject[subject][segment][finger] = []

                for run in run_list[subject]:
                    abs_diff_tps[subject][run][segment][finger] = []




for subject in subject_list:   
    for run in run_list[subject]:
        for segment in segment_list:
            time_current =  np.array(data_segments[subject][run][sensor][segment]['rel_time']/1000) #CONVERSION EN SECONDES
            
            for finger in fingers_cal:  
                
                data_current = np.array(data_segments[subject][run][sensor][segment][finger])
                #Newton diff method
                diff_tps = (data_current[1:-1] - data_current[0:-2])/(time_current[1:-1] - time_current[0:-2])
                #take abs value of derivative to take into account press and releas variations
                abs_diff_tps[subject][run][segment][finger] = abs(diff_tps)
                # compute range value to have measurement of how much force is applied
                range_raw_tps = max(data_current) - min(data_current)
                
                tps_segment_subject[subject][segment][finger].append(abs(diff_tps)) #valeurs abs derivative tps a la suite pour les runs de 1 sujet, 1 finger
                range_raw_tps_subject[subject][segment][finger].append(range_raw_tps) #valeurs des range a la suite pour les runs de 1 sujet pour 1 finger
                

            df = pd.DataFrame({
                fingers_cal[0]: data_segments[subject][run][sensor][segment][fingers_cal[0]][1:-1],
                fingers_cal[1]: data_segments[subject][run][sensor][segment][fingers_cal[1]][1:-1],
                fingers_cal[2]: data_segments[subject][run][sensor][segment][fingers_cal[2]][1:-1],
                fingers_cal[3]: data_segments[subject][run][sensor][segment][fingers_cal[3]][1:-1],
                fingers_cal[4]: data_segments[subject][run][sensor][segment][fingers_cal[4]][1:-1],
                fingers_cal[5]: data_segments[subject][run][sensor][segment][fingers_cal[5]][1:-1],
                'Derivation, abs value, left index':abs_diff_tps[subject][run][segment][fingers_cal[0]],
                'Derivation, abs value, left middle':abs_diff_tps[subject][run][segment][fingers_cal[1]],
                'Derivation, abs value, left thumb':abs_diff_tps[subject][run][segment][fingers_cal[2]],
                'Derivation, abs value, right index':abs_diff_tps[subject][run][segment][fingers_cal[3]],
                'Derivation, abs value, right middle':abs_diff_tps[subject][run][segment][fingers_cal[4]],
                'Derivation, abs value, right thumb':abs_diff_tps[subject][run][segment][fingers_cal[5]]
            })
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=df[fingers_cal[0]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[0]))
            fig.add_trace(go.Scatter(y=df[fingers_cal[1]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[1]))
            fig.add_trace(go.Scatter(y=df[fingers_cal[2]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[2]))

            fig.add_trace(go.Scatter(y=df[fingers_cal[3]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[3]))
            fig.add_trace(go.Scatter(y=df[fingers_cal[4]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[4]))
            fig.add_trace(go.Scatter(y=df[fingers_cal[5]], x=time_current-time_current[0],
                    mode='lines', name=fingers_cal[5]))
            #fig.show()
            
    

            if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION', 'tps_plots' )):
                os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION', 'tps_plots'))
            fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION','tps_plots', 'segment' + segment  +'_raw_cal.png'))
    
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, left index'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, left index'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, left middle'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, left middle'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, left thumb'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, left thumb'))

            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right index'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, right index'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right middle'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, right middle'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right thumb'], x=time_current-time_current[0],
                    mode='lines', name='Derivation, abs value, right thumb'))
            #fig.show()
            
            fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'SENSORS', 'SEGMENTATION','tps_plots', 'segment' + segment +'_abs_derivative.png'))

            

### plot tps colormap for each run and for every run of 1 subject

In [39]:
sensor = 'TPS_calibrated'
for subject in subject_list:   
    for segment in segment_list:
        
            fig = go.Figure()
            x_values = []
            y_values_tot = []
            y_min = 0
            y_limit_diff = 1.5
            y_limit_range = 1.5
            y_max_range = 2
            y_max_diff = 0.5

        
            for i, finger in enumerate(fingers_cal):
                x_values.append(i)
                
                tot_signal = np.concatenate(tps_segment_subject[subject][segment][finger]) #tous les runs pour un subject, valeur abs de la derivee
                y_values_tot.append(np.mean(tot_signal)) #valeur moyenne pour un sujet
            
            y_values_tot = (np.array(y_values_tot) - y_min) / (y_max_diff - y_min)

            fig.add_trace(go.Scatter(
                x=x_values,
                y=y_values_tot,
                marker=dict(
                    size=16,
                    cmax=1,
                    cmin=0,
                    color=y_values_tot,
                    colorbar=dict(
                        title='Colorbar, 1=' + str(y_max_diff) + '[lbs/s]'
                    ),
                    colorscale="rainbow"
                ),
                mode="markers"))
            
            fig.update_layout(title='Mean abs value of derivative, one subject',
                   xaxis_title='left_index,  left_middle,  left_thumb, right_index, right_middle, right_thumb',
                   yaxis_title='TPS data',
                   yaxis_range=[0,y_limit_diff])
            
            if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, 'PLOTS')):
                    os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, 'PLOTS'))

            fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, 'PLOTS', 'TPS_diff_colormap_subject' +subject + '_segment' + segment + '_' +'.png'))
            
          
            fig = go.Figure()
            x_values = []
            y_values_tot = []

            for i, finger in enumerate(fingers_cal):
                x_values.append(i)
                y_values_tot.append(np.mean(range_raw_tps_subject[subject][segment][finger])) #ajoute a y les valeurs moyennes des range des run pour chaque doigt

            y_values_tot = (np.array(y_values_tot) - y_min) / (y_max_range - y_min)

            fig.add_trace(go.Scatter(
                x=x_values,
                y=y_values_tot,
                marker=dict(
                    size=16,
                    cmax=1,
                    cmin=0,
                    color=y_values_tot,
                    colorbar=dict(
                        title='Colorbar, 1=' + str(y_max_range) + '[lbs]'
                    ),
                    colorscale="rainbow"
                ),
                mode="markers"))
            
            fig.update_layout(title='Mean value of ranges, one subject',
                   xaxis_title='left_index,  left_middle,  left_thumb, right_index, right_middle, right_thumb',
                   yaxis_title='TPS data',
                   yaxis_range=[0,y_limit_range])

            fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, 'PLOTS', 'TPS_range_colormap_subject' +subject + '_segment' + segment + '_' +'.png'))


     

      
                
            #figures for each run separately
            for j, run in enumerate(run_list[subject]):
                fig = go.Figure()
                x_values = []
                y_values = []      
            
                
                for i, finger in enumerate(fingers_cal):
                    x_values.append(i)

                    mean_signal = np.mean(tps_segment_subject[subject][segment][finger][j]) #moyenne pour 1 run pour un subject, valeur abs de la derivee
                    y_values.append(mean_signal)

                y_values = (np.array(y_values) - y_min) / (y_max_diff - y_min)
                
                

                fig.add_trace(go.Scatter(
                    x=x_values,
                    y=y_values,
                    marker=dict(
                        size=16,
                        cmax=1,
                        cmin=0,
                        color=y_values,
                        colorbar=dict(
                            title='Colorbar, 1=' + str(y_max_diff) + '[lbs/s]'
                        ),
                        colorscale="rdylgn_r"
                    ),
                    mode="markers"))
                
                fig.update_layout(title='Mean abs value of derivative, one run',
                   xaxis_title='left_index,  left_middle,  left_thumb, right_index, right_middle, right_thumb',
                   yaxis_title='TPS data',
                   yaxis_range=[0,y_limit_diff])


                if not os.path.exists(join(path_to_data_folder, recording_session[subject] , subject, run, 'PLOTS')):
                            os.makedirs(join(path_to_data_folder, recording_session[subject] , subject, run, 'PLOTS'))

                fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'PLOTS', 'TPS_diff_colormapsubject' +subject + '_segment' + segment + 'run_' + run +'.png'))

                
            for j, run in enumerate(run_list[subject]):
                fig = go.Figure()
                x_values = []
                y_values = []

                for i, finger in enumerate(fingers_cal):
                    x_values.append(i)
                    y_values.append(np.array(range_raw_tps_subject[subject][segment][finger][j]))
                    
                y_values = (np.array(y_values) - y_min) / (y_max_range - y_min)

                fig.add_trace(go.Scatter(
                    x=x_values,
                    y=y_values,
                    marker=dict(
                        size=16,
                        cmax=1,
                        cmin=0,
                        color=y_values,
                        colorbar=dict(
                            title='Colorbar, 1=' + str(y_max_range) + '[lbs]'
                        ),
                        colorscale="rdylgn_r"
                    ),
                    mode="markers"))
                
                fig.update_layout(title='Values of ranges, one run',
                   xaxis_title='left_index,  left_middle,  left_thumb, right_index, right_middle, right_thumb',
                   yaxis_title='TPS data',
                   yaxis_range=[0,y_limit_range])


                fig.write_image(join('..', 'data_exp', recording_session[subject] , subject, run, 'PLOTS', 'TPS_range_colormapsubject' +subject + '_segment' + segment + 'run_' + run +'.png'))


                 

In [87]:
print(y_values)

0.6019494491256694


In [105]:
print(fingers_cal)

['elem 1 calibrated left_index', 'elem 0 calibrated left_middle', 'elem 2 calibrated left_thumb', 'elem 7 calibrated right_index', 'elem 8 calibrated right_middle', 'elem 6 calibrated right_thumb']
